## data storm 4.0 group 095

### import needed libraries

In [1]:
import pandas as pd
import numpy as nps

### import data into code

In [2]:
transaction_data=pd.read_csv("../data/Historical-transaction-data.csv")
store_data=pd.read_csv("../data/Store-info.csv")

get the first few rows of the transaction_data and store_data

In [3]:
store_data.head()

,shop_id,shop_area_sq_ft,shop_profile
0,SHOP047,528,Moderate
1,SHOP009,676,High
2,SHOP083,676,Low
3,SHOP117,676,Low
4,SHOP042,676,Low


In [4]:
transaction_data.head()

,item_description,transaction_date,invoice_id,customer_id,shop_id,item_price,quantity_sold
0,ORANGE BARLEY 1.5L,2021-12-11T00:00:00.000Z,147.0,BGXA,SHOP008,220,2
1,GINGER BEER 1.5L,2021-10-17T00:00:00.000Z,371.0,IA25,SHOP112,220,2
2,TONIC PET 500ML,2021-12-13T00:00:00.000Z,484.0,VN7V,SHOP008,160,2
3,CREAM SODA 1L,2021-12-13T00:00:00.000Z,484.0,VN7V,SHOP008,150,2
4,STRAWBERRY MILK 180ML,2021-10-23T00:00:00.000Z,1310.0,7S00,SHOP112,210,5


## Data pre-processing

### remove unwnated data and get the important data

In this step,we have to extract the details from the transaction_data.we think most important feature is the price. we also have to consider the number of items,the number of customers,number of different items sold

we know that each row should be unique becuase if shop buy same product then quantity_sold should be increased.we think if duplicate row means there is a problem in data entering.data set should be consistent.so we have to remove duplicate values

In [5]:
duplicate_invoice=transaction_data.duplicated().sum()
print("number of duplicate invoices = ",duplicate_invoice)

number of duplicate invoices =  31211


In [6]:
print("duplicate columns as percentage = ",duplicate_invoice/transaction_data.shape[0]*100)

duplicate columns as percentage =  6.58496035647525


In [7]:
# to drop the duplicates
transaction_data=transaction_data.drop_duplicates()

to group by shop_id

In [8]:
grouped_transaction_data = transaction_data.groupby('shop_id')

In [9]:
grouped_transaction_data.head()

,item_description,transaction_date,invoice_id,customer_id,shop_id,item_price,quantity_sold
0,ORANGE BARLEY 1.5L,2021-12-11T00:00:00.000Z,147.0,BGXA,SHOP008,220,2
1,GINGER BEER 1.5L,2021-10-17T00:00:00.000Z,371.0,IA25,SHOP112,220,2
2,TONIC PET 500ML,2021-12-13T00:00:00.000Z,484.0,VN7V,SHOP008,160,2
3,CREAM SODA 1L,2021-12-13T00:00:00.000Z,484.0,VN7V,SHOP008,150,2
4,STRAWBERRY MILK 180ML,2021-10-23T00:00:00.000Z,1310.0,7S00,SHOP112,210,5
...,...,...,...,...,...,...,...
5031,NaN,2021-10-15T00:00:00.000Z,1471067.0,W8QK,SHOP072,70,1
5032,CREAM SODA 500ML,2021-10-16T00:00:00.000Z,1471404.0,6JBA,SHOP072,100,2
5035,CREAM SODA 500ML,2021-10-16T00:00:00.000Z,1471404.0,6JBA,SHOP072,100,0
5036,GINGER BEER 1.5L,2021-10-16T00:00:00.000Z,1471453.0,0XKY,SHOP072,220,2


### to get the number of unique values for given column

In [10]:
for col in transaction_data.columns:
    print(col," number of unique values = ",transaction_data[col].nunique())

item_description  number of unique values =  37
transaction_date  number of unique values =  62
invoice_id  number of unique values =  343342
customer_id  number of unique values =  191636
shop_id  number of unique values =  124
item_price  number of unique values =  209
quantity_sold  number of unique values =  69


In [11]:
# transaction_data_SHOP008 = transaction_data[transaction_data['shop_id'] == 'SHOP008']
# transaction_data_SHOP008.shape

### to get the number of missing values for each colum

In [12]:
for col in transaction_data.columns:
    print(col," number of missing values = ",transaction_data[col].isna().sum())

item_description  number of missing values =  35066
transaction_date  number of missing values =  0
invoice_id  number of missing values =  5769
customer_id  number of missing values =  0
shop_id  number of missing values =  0
item_price  number of missing values =  0
quantity_sold  number of missing values =  0


so , that we don't need to use any missing value technic to haddle the missing values in the dataset because we only need price as a multiplication of item_price and quantity_sold. we can also take number of quatity sold by each shop without any problem.we can also take the unique number of customers for each shop because this column also don't have any missing values.

we have to take the number of unique items for each shop.but there are some missing values for that column.but we need only the number of unique values.if we fill the missing values.sometimes,this give an error.so we don't fill item_description column

to get the number of unique shop ids 

In [13]:
unique_shop_id_names = store_data['shop_id'].unique()
print("number of unique shops = ",store_data['shop_id'].nunique())

number of unique shops =  124


we have to find the number of missing values for each column in store_data

In [14]:
for col in store_data.columns:
    print(col," number of missing values = ",store_data[col].isna().sum())

shop_id  number of missing values =  0
shop_area_sq_ft  number of missing values =  0
shop_profile  number of missing values =  24


we can see number of missing values shop_id and shop_area_sq_ft is zero.so,we don't need to use handling misssing values techniques because last shop ids are given to predict

we have to get the get the ids of shops to train and predict

In [15]:
train_shop_id_names=[]
predict_shop_id_names=[]
for ind in store_data.index:
    if(pd.isna(store_data['shop_profile'][ind])):
        predict_shop_id_names+=[store_data["shop_id"][ind]]
    else:
        train_shop_id_names+=[store_data["shop_id"][ind]]
print(store_data["shop_profile"][ind])

nan


In [16]:
print("number of shop_id for train = ",len(train_shop_id_names))
print("number of shop_id for predict = ",len(predict_shop_id_names))

number of shop_id for train =  100
number of shop_id for predict =  24


###  create new dataset using previous data

we have to build new  columns using the transaction_data dataset

they are, num_unique_items,
          num_unique_customers,
          quantity_items,
          total_price
          
we also have to add shop_id,shop_area_sq_ft,shop_profile

In [24]:
# to create column for total for item
transaction_data["total_for_items"]=transaction_data["quantity_sold"]*transaction_data["item_price"]

In [25]:
col_names_x=['shop_id','num_unique_items','num_unique_customers','quantity_items','total_price','shop_area_sq_ft']

main_data_x=pd.DataFrame(columns=col_names_x)
main_data_y=pd.DataFrame(columns=['shop_id',"shop_profile"])

In [26]:
for shop_id in train_shop_id_names:
    #print(shop_id)
    data_shop=transaction_data[transaction_data['shop_id'] == shop_id]
    #print(data_shop.head())
    num_unique_items=data_shop["item_description"].nunique()
    num_unique_customers=data_shop["customer_id"].nunique()
    quantity_items=data_shop["quantity_sold"].nunique()
    total_price=data_shop["total_for_items"].sum()
    #print("num_unique_items = ",num_unique_items,' num_unique_customers = ',num_unique_customers," quantity_items = ",quantity_items," total_price =",total_price)
    store_data_new=store_data[store_data['shop_id'] == shop_id]
    shop_area_sq_ft=store_data_new["shop_area_sq_ft"].iloc[0]
    shop_profile=store_data_new["shop_profile"].iloc[0]
    #print("shop_area_sq_ft = ",shop_area_sq_ft,' shop_profile = ',shop_profile)
    new_row_x={'shop_id':shop_id,'num_unique_items':num_unique_items,'num_unique_customers':num_unique_customers,'quantity_items':quantity_items,'total_price':total_price,'shop_area_sq_ft':shop_area_sq_ft}
    new_row_y={'shop_id':shop_id,'shop_profile':shop_profile}
    main_data_x=pd.concat([main_data_x, pd.DataFrame(new_row_x, index=[0])], ignore_index=True)
    main_data_y=pd.concat([main_data_y, pd.DataFrame(new_row_y, index=[0])], ignore_index=True)
    
    

make index as the "shop_id"

In [27]:
# main_data_x.set_index("shop_id", inplace=True)
# main_data_y.set_index("shop_id", inplace=True)
main_data_x = main_data_x.drop("shop_id", axis=1)
main_data_y = main_data_y.drop("shop_id", axis=1)

In [28]:
main_data_x.head()

,num_unique_items,num_unique_customers,quantity_items,total_price,shop_area_sq_ft
0,35,928,14,802525,528
1,35,2498,18,1891660,676
2,36,1900,17,1614960,676
3,35,2037,19,2235870,676
4,35,1841,16,1276790,676


#### we know that Moderate is better than Low.High is better than Moderate.so that "shop_profile" is an ordinal data

In [32]:
main_data_y["shop_profile"].unique()

array(['Moderate', 'High', 'Low'], dtype=object)

we replace Low=0,Moderate=1,High=2

In [33]:
main_data_y["shop_profile"] = pd.Categorical(main_data_y["shop_profile"], categories=['Low', 'Moderate', 'High'], ordered=True).codes

### import neeeded libraries for testing

In [34]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

#### Split the dataset into training and testing sets

In [35]:
X_train, X_test, y_train, y_test = train_test_split(main_data_x, main_data_y, test_size=0.3, random_state=42)

#### for testing purpose we need only one column.so that get only one column

In [36]:
y_train = y_train['shop_profile'].values.ravel()
y_test = y_test['shop_profile'].values.ravel()

### Random Forest Classifier

In [37]:
# Choose hyperparameters
n_estimators = 200
max_depth = 50


# Train the model
model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
model.fit(X_train, y_train)


# Evaluate the model
score = model.score(X_test, y_test)
print("Accuracy using model: {:.2f}%".format(score * 100))

# Predict with the model
y_pred = model.predict(X_test)

f1 = f1_score(y_test, y_pred, average='weighted')

print('F1 score for random forest :', f1)

Accuracy using model: 76.67%
F1 score for random forest : 0.7673486786018755


In [38]:
y_pred

array([0, 2, 2, 0, 1, 1, 2, 0, 2, 1, 1, 2, 0, 0, 1, 0, 2, 1, 0, 0, 2, 2,
       0, 0, 1, 0, 0, 1, 1, 1], dtype=int8)

In [39]:
y_test

array([2, 2, 2, 0, 1, 0, 2, 0, 1, 1, 1, 2, 1, 0, 1, 0, 2, 2, 0, 0, 2, 2,
       1, 0, 1, 0, 0, 0, 1, 1], dtype=int8)

### KNN alorithm

In [40]:
knn = KNeighborsClassifier(n_neighbors=1)

# train the model
knn.fit(X_train, y_train)

# predict the classes for the test set
y_pred = knn.predict(X_test)

# calculate the accuracy of the model
f1 = f1_score(y_test, y_pred, average='weighted')

print('F1 score:', f1*100)

F1 score: 60.585748792270536
